# Подготовка ноутбука 

## Пробрасываем magic methods

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

## Imports

In [ ]:
import pandas as pd 
import numpy as np 
from utils.s3 import s3_download_file, get_s3_client, s3_list_files, s3_upload_file, s3_upload_folder
from utils.zip import extract_zip_advanced
from utils.useful_functions import extract_coordinates, move_and_remove_files, merge_tables_with_tolerance
from dotenv import load_dotenv
import os 
from pathlib import Path
from warnings import filterwarnings 
import torch 
import torchvision
from IPython.display import display
import shutil

## Нужные переменные 

In [ ]:
ROOT_DIR = Path('../../')
load_dotenv()
filterwarnings(action='ignore')

# Загрузка датафрейма

## Пробрасываем креды

In [ ]:
get_s3_client()

## Смотрим, что есть в хранилище 

In [ ]:
s3_list_files()

## Загружаем из s3

### Загружаем тренировочный датасет 

In [ ]:
s3_download_file(file_s3_src='raw_data/Датасет.zip', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/data.zip', 
                 bucket_name='s3-dvc',)

In [ ]:
s3_download_file(file_s3_src='raw_data/Объекты недвижимости, не соответствующие градостроительным нормам_00-022_Август.zip', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/build_no_norm_aug.zip', 
                 bucket_name='s3-dvc',)

In [ ]:
s3_download_file(file_s3_src='raw_data/Объекты недвижимости, не соответствующие градостроительным нормам_00-022_Июль.zip', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/build_no_norm_jule.zip', 
                 bucket_name='s3-dvc',)

In [ ]:
s3_download_file(file_s3_src='raw_data/Строительная площадка_18-001_Август.zip', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/building_aug.zip', 
                 bucket_name='s3-dvc',)

In [ ]:
s3_download_file(file_s3_src='raw_data/Строительная площадка_18-001_Июль.zip', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/building_jule.zip', 
                 bucket_name='s3-dvc',)

### Загружаем таргет 

In [ ]:
s3_download_file(file_s3_src='raw_data/1498CSV.zip', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/target_build.zip', 
                 bucket_name='s3-dvc',)

In [ ]:
s3_download_file(file_s3_src='raw_data/Реестр камер подъездного видеонаблюдения версия 3.30.zip', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/target_garb.zip', 
                 bucket_name='s3-dvc',)

In [ ]:
s3_download_file(file_s3_src='raw_data/2386XLSX.zip', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/many_people_target.zip', 
                 bucket_name='s3-dvc',)

In [ ]:
s3_download_file(file_s3_src='raw_data/Объекты недвижимости, не соответствующие градостроительным нормам_00-022_Август.xlsx', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/build_no_norm_aug_target.xlsx', 
                 bucket_name='s3-dvc',)

In [ ]:
s3_download_file(file_s3_src='raw_data/Объекты недвижимости, не соответствующие градостроительным нормам_00-022_Июль.xlsx', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/build_no_norm_jule_target.xlsx', 
                 bucket_name='s3-dvc',)

In [ ]:
s3_download_file(file_s3_src='raw_data/Строительная площадка_18-001_Август.xlsx', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/building_aug_target.xlsx', 
                 bucket_name='s3-dvc',)

In [ ]:
s3_download_file(file_s3_src='raw_data/Строительная площадка_18-001_Июль.xlsx', 
                 file_local_dst=ROOT_DIR / 'data/raw_data/building_jule_target.xlsx', 
                 bucket_name='s3-dvc',)

## Извлекаем из zip

### Извлекаем тренировочный датасет 

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/build_no_norm_aug.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/build_no_norm_aug',
                     remove_after_extract=True,)

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/build_no_norm_jule.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/build_no_norm_jule',
                     remove_after_extract=True,)

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/building_aug.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/building_aug',
                     remove_after_extract=True,)

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/building_jule.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/building_jule',
                     remove_after_extract=True,)

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/data.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/data',
                     remove_after_extract=True,)

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/data/Датасет/Любые объекты.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/data/any_objects',
                     remove_after_extract=True,)

# После загрузки мы перенесем фотки из ïεíδÑ «íΩÑ¬Γδ в корневую any_objects
source_dir = ROOT_DIR / 'data/raw_data/data/any_objects/ïεíδÑ «íΩÑ¬Γδ'
destination_dir = ROOT_DIR / 'data/raw_data/data/any_objects/'

move_and_remove_files(source_dir=source_dir, 
                      destination_dir=destination_dir,
                      remove_after_move=True)


In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/data/Датасет/Метаданные.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/data/metadata',
                     remove_after_extract=True,)

# Переносим в корневую metadata
destination_dir = ROOT_DIR / 'data/raw_data/data/metadata'

move_and_remove_files(source_dir=ROOT_DIR / 'data/raw_data/data/metadata/ïεíδÑ «íΩÑ¬Γδ/è¿¡ß', 
                      destination_dir=destination_dir,)
move_and_remove_files(source_dir=ROOT_DIR / 'data/raw_data/data/metadata/ïεíδÑ «íΩÑ¬Γδ/êìæ', 
                      destination_dir=destination_dir,)

shutil.move(src=ROOT_DIR / 'data/raw_data/data/metadata/îÑΓáñá¡¡δÑ/êìæ.zip',
            dst=ROOT_DIR / 'data/raw_data/data/metadata')
shutil.move(src=ROOT_DIR / 'data/raw_data/data/metadata/îÑΓáñá¡¡δÑ/è¿¡ß.zip',
            dst=ROOT_DIR / 'data/raw_data/data/metadata')
# Удаляем ненужную îÑΓáñá¡¡δÑ
shutil.rmtree(ROOT_DIR / 'data/raw_data/data/metadata/îÑΓáñá¡¡δÑ')

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/data/Датасет/Пример снимков БПЛА с ббох.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/data/bpla',
                     remove_after_extract=True,)

# После загрузки мы перенесем фотки из Åα¿¼Ñα ß¡¿¼¬«ó üÅïÇ ß íí«σ в корневую bpla
source_dir = ROOT_DIR / 'data/raw_data/data/bpla/Åα¿¼Ñα ß¡¿¼¬«ó üÅïÇ ß íí«σ'
destination_dir = ROOT_DIR / 'data/raw_data/data/bpla'

move_and_remove_files(source_dir=source_dir, 
                      destination_dir=destination_dir,
                      remove_after_move=True)

In [ ]:
shutil.rmtree(ROOT_DIR / 'data/raw_data/data/Датасет')

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/data/metadata/êìæ.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/data/metadata/êìæ',
                     remove_after_extract=True,)
# Переименовываем в нормальные названия 
os.rename(src=ROOT_DIR / 'data/raw_data/data/metadata/êìæ',
          dst=ROOT_DIR / 'data/raw_data/data/metadata/INC')

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/data/metadata/è¿¡ß.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/data/metadata/è¿¡ß',
                     remove_after_extract=True,)
os.rename(src=ROOT_DIR / 'data/raw_data/data/metadata/è¿¡ß',
          dst=ROOT_DIR / 'data/raw_data/data/metadata/Kins')

### Извлекаем таргет 

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/target_build.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/target_build',
                     remove_after_extract=True,)

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/target_garb.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/target_garb',
                     remove_after_extract=True,)

In [ ]:
extract_zip_advanced(zip_path=ROOT_DIR / 'data/raw_data/many_people_target.zip',
                     extract_to=ROOT_DIR / 'data/raw_data/many_people_target',
                     remove_after_extract=True,)

# Объединяем данные 

## Выгружаем камеры в местах большого скопления народа

In [ ]:
target_many_people = pd.read_excel(io=ROOT_DIR / 'data/raw_data/many_people_target/data-2386-2025-07-24.xlsx',
                                   usecols=['ID', 'geoData'])[1:]

In [ ]:
# Извлекаем координаты и создаем новые колонки
target_many_people[['longitude', 'latitude']] = target_many_people['geoData'].apply(
    lambda x: pd.Series(extract_coordinates(x))
)
target_many_people.drop('geoData', axis=1, inplace=True)

##  Выгружаем дворовые камеры с таргетом 

In [ ]:
builging_echd = pd.read_excel(io=ROOT_DIR / 'data/raw_data/data/metadata/INC/18-001_gin_building_echd_19.08.25.xlsx',
                            usecols=['Имя файла', 'longitude', 'latitude',])

In [ ]:
target_build = pd.read_csv(filepath_or_buffer=ROOT_DIR / 'data/raw_data/target_build/data-1498-2024-11-25.csv',
                           usecols=['geoData', 'ID'],
                           sep=';',).iloc[1:]

In [ ]:
# Извлекаем координаты и создаем новые колонки
target_build[['longitude', 'latitude']] = target_build['geoData'].apply(
    lambda x: pd.Series(extract_coordinates(x))
)
target_build.drop('geoData', axis=1, inplace=True)

##  Выгружаем подъездные камеры с таргетом

In [ ]:
garbage_echd = pd.read_excel(io=ROOT_DIR / 'data/raw_data/data/metadata/INC/19-001_gin_garbage_echd_19.08.25.xlsx',
                              usecols=['Имя файла', 'longitude', 'latitude',])

In [ ]:
target_garb = pd.read_excel(io=ROOT_DIR / 'data/raw_data/target_garb/data-1500-2025-07-28.xlsx',
                           usecols=['geoData', 'ID',],
                           ).iloc[1:]

In [ ]:
# Извлекаем координаты и создаем новые колонки
target_garb[['longitude', 'latitude']] = target_garb['geoData'].apply(
    lambda x: pd.Series(extract_coordinates(x))
)
target_garb.drop('geoData', axis=1, inplace=True)

## Выгружаем новые данные

In [ ]:
build_no_norm_aug_target = pd.read_excel(ROOT_DIR / "data/raw_data/build_no_norm_aug_target.xlsx",
                                         usecols=['Имя файла', 'latitude', 'longitude',]
                                         )

In [ ]:
build_no_norm_aug_target = pd.read_excel(ROOT_DIR / "data/raw_data/build_no_norm_jule_target.xlsx",
                                         usecols=['Имя файла', 'latitude', 'longitude',]
                                         )

In [ ]:
build_no_norm_aug_target = pd.read_excel(ROOT_DIR / "data/raw_data/building_aug_target.xlsx",
                                         usecols=['Имя файла', 'latitude', 'longitude',]
                                         )

In [ ]:
build_no_norm_aug_target = pd.read_excel(ROOT_DIR / "data/raw_data/building_jule_target.xlsx",
                                         usecols=['Имя файла', 'latitude', 'longitude',]
                                         )

In [ ]:
build_target_data = pd.concat([build_no_norm_aug_target, build_no_norm_aug_target,
                         build_no_norm_aug_target, build_no_norm_aug_target], 
                         axis=0)

In [ ]:
os.mkdir(ROOT_DIR / 'data/processed_data/')
build_target_data.to_csv(path_or_buf=ROOT_DIR / 'data/processed_data/build_target_data.csv',
            index=False,)

## Объединяем фотографии


In [ ]:
destination_dir = ROOT_DIR / 'data/processed_data/images_build/'
os.mkdir(destination_dir)
os.mkdir(ROOT_DIR / 'data/processed_data/merge_images/')

In [ ]:
move_and_remove_files(source_dir=ROOT_DIR / 'data/raw_data/build_no_norm_aug/Объекты недвижимости, не соответствующие градостроительным нормам_00-022_Август', 
                      destination_dir=destination_dir,
                      remove_after_move=True)

In [ ]:
move_and_remove_files(source_dir=ROOT_DIR / 'data/raw_data/build_no_norm_jule/Объекты недвижимости, не соответствующие градостроительным нормам_00-022_Июль', 
                      destination_dir=destination_dir,
                      remove_after_move=True)

In [ ]:
move_and_remove_files(source_dir=ROOT_DIR / 'data/raw_data/building_aug/Строительная площадка_18-001_Август', 
                      destination_dir=destination_dir,
                      remove_after_move=True)

In [ ]:
move_and_remove_files(source_dir=ROOT_DIR / 'data/raw_data/building_jule/Строительная площадка_18-001_Июль', 
                      destination_dir=destination_dir,
                      remove_after_move=True)

## Объединяем данные

In [ ]:
# Называю real так как это данные которые передали нам орги 
real_data = pd.concat([garbage_echd, builging_echd], axis=0)
display(real_data.shape)
real_data.head()

In [ ]:
# Называю target так как это данные которые мы выгрузили из открытых источников
# На них id камеры (нам нужно это для оптимизации алгоритма)
target_data = pd.concat([target_build, target_garb, target_many_people], axis=0)
display(target_data.shape)
target_data.head()

In [ ]:
data = merge_tables_with_tolerance(target_data, real_data, max_distance_meters=100).drop_duplicates()

In [ ]:
data.head()

In [ ]:
data.to_csv(path_or_buf=ROOT_DIR / 'data/processed_data/merged_data.csv',
            index=False,)

# Перемещяем все фотки в единую папку 

In [ ]:
os.mkdir(ROOT_DIR / 'data/raw_data/data/metadata/INC/united_image')

In [ ]:
move_and_remove_files(source_dir=ROOT_DIR / 'data/raw_data/data/metadata/INC/18-001_gin_building_echd_19.08.25', 
                      destination_dir=ROOT_DIR / 'data/raw_data/data/metadata/INC/united_image',
                      remove_after_move=True,)

In [ ]:
move_and_remove_files(source_dir=ROOT_DIR / 'data/raw_data/data/metadata/INC/19-001_gin_garbage_echd_19.08.25', 
                      destination_dir=ROOT_DIR / 'data/raw_data/data/metadata/INC/united_image',
                      remove_after_move=True,)

In [ ]:
move_and_remove_files(source_dir=ROOT_DIR / 'data/raw_data/data/metadata/INC/united_image', 
                      destination_dir=ROOT_DIR / 'data/processed_data/merge_images/',
                      remove_after_move=False)

# Загружаем в s3

## Пробрасываем конект

In [ ]:
get_s3_client()

## Передаем в s3

In [ ]:
s3_upload_file(file_local_src=ROOT_DIR / 'data/processed_data/merged_data.csv',
               file_s3_dst='processed_data/merge_data.csv',
               bucket_name='s3-dvc',)

In [ ]:
s3_upload_file(file_local_src=ROOT_DIR / 'data/processed_data/build_target_data.csv',
               file_s3_dst='processed_data/build_target_data.csv',
               bucket_name='s3-dvc',)

In [ ]:
s3_upload_folder(local_folder=ROOT_DIR / 'data/processed_data/merge_images/',
               s3_prefix='processed_data/merged_images',
               bucket_name='s3-dvc',)

In [ ]:
s3_upload_folder(local_folder=ROOT_DIR / 'data/processed_data/images_build',
               s3_prefix='processed_data/images_build',
               bucket_name='s3-dvc',)
# последний файл c201fc12-425d-4ecf-8a89-9dd4fc0fb3aa.jpg